## ;-;

In [1]:
import dotenv
import os
import sys
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


dotenv.load_dotenv()

asdEdfPath = os.getenv("ASD_EDF_DIR_PATH")
ncEdfPath = os.getenv("NC_EDF_DIR_PATH")

asdNpyPath = os.getenv("ASD_NUMPY_DIR_PATH")
ncNpyPath = os.getenv("NC_NUMPY_DIR_PATH")

asdfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/asd.fif"
ncfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/nc.fif"

In [4]:
import os
import numpy as np
import mne



# EEG parameters
sfreq = 256  # Sampling rate
n_channels = 20  # 19 electrodes + 1 ground
epoch_length = 10  # Epoch length in seconds
epoch_samples = epoch_length * sfreq  # Samples per epoch (2560)

# Preprocessing parameters
t_start = 2  # Start trimming (in seconds)
t_duration = 300  # Use 5 minutes of data
low_cutoff, high_cutoff = 0.5, 40  # Bandpass filter range

# Channel names and types
ch_names = [f"EEG{i}" for i in range(19)]  # Remove ground channel from list
ch_types = ["eeg"] * 19  # Only EEG channels

def preprocess_and_save(data_path, output_filename):
    all_epochs = []
    
    for file in os.listdir(data_path):
        if file.endswith(".npy"):
            file_path = os.path.join(data_path, file)
            raw_data = np.load(file_path)  # Shape: (20, N)
            
            # Extract time samples
            n_samples = raw_data.shape[1]
            start_sample = t_start * sfreq
            end_sample = start_sample + (t_duration * sfreq)
            
            if end_sample > n_samples:
                print(f"Skipping {file}, not enough data")
                continue
            
            data = raw_data[:, start_sample:end_sample]  # Trimmed data
            
            # Remove the last channel (GND)
            data = data[:-1, :]
            
            # Create MNE RawArray
            info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
            raw = mne.io.RawArray(data, info)
            
            # Apply bandpass filter
            raw.filter(l_freq=low_cutoff, h_freq=high_cutoff, fir_design='firwin')
            
            # Epoching (10-sec non-overlapping)
            events = np.array([[i * epoch_samples, 0, 1] for i in range(t_duration // epoch_length)])
            epochs = mne.Epochs(raw, events, event_id=1, tmin=0, tmax=epoch_length, baseline=None, detrend=1, preload=True)
            
            # Verify epochs shape
            print(f"Epoch shape for {file}: {epochs.get_data().shape}")
            
            all_epochs.append(epochs)

    if all_epochs:
        merged_epochs = mne.concatenate_epochs(all_epochs)
        merged_epochs.save(output_filename, overwrite=True)
    else:
        print(f"No valid data in {data_path}")


# Process ASD and NC groups
preprocess_and_save(asdNpyPath, "asd.fif")
preprocess_and_save(ncNpyPath, "nc.fif")


Creating RawArray with float64 data, n_channels=19, n_times=76800
    Range : 0 ... 76799 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 2561 original time points ...
1 bad epochs dropped
Epoch shape for CASD02.npy: (29, 19, 2561)
Creating RawArray with float64 data, n_channels=19, n_times

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Epoch shape for CASD17.npy: (29, 19, 2561)
Creating RawArray with float64 data, n_channels=19, n_times=76800
    Range : 0 ... 76799 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 2561 original time points ...
1 bad epochs dropped
Epoch shape for CASD03.npy: (29, 19, 2561)
Creating RawArra

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 2561 original time points ...
1 bad epochs dropped
Epoch shape for CASD29.npy: (29, 19, 2561)
Creating RawArray with float64 data, n_channels=19, n_times=76800
    Range : 0 ... 76799 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)

Not setting metadata
30 matching events found
No baseline c

...

Epoch shape for CNC10.npy: (29, 19, 2561)
Creating RawArray with float64 data, n_channels=19, n_times=76800
    Range : 0 ... 76799 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 2561 original time points ...
1 bad epochs dropped
Epoch shape for CNC04.npy: (29, 19, 2561)
Creating RawArray 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Epoch shape for CNC11.npy: (29, 19, 2561)
Not setting metadata
986 matching events found
No baseline correction applied
Overwriting existing file.


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/1714301843.py:62: RuntimeWarning: This filename (nc.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  merged_epochs.save(output_filename, overwrite=True)


Overwriting existing file.


## Feature Extraction

### 1. Band Power

In [9]:
import dotenv
import os
import sys
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from brainasd.features.psd import EEGPowerSpectrum


dotenv.load_dotenv()

asdfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/asd.fif"
ncfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/nc.fif"
allFeaturesPath = os.getenv("ALL_FEATURES_DIR_PATH")

NUM_CHANNELS = 19
EEG_SFREQ = 256


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }  

In [ ]:
asdEpochs = mne.read_epochs(asdfilteredEpochsPath)
ncEpochs = mne.read_epochs(ncfilteredEpochsPath)

asdData = asdEpochs.get_data()
ncData = ncEpochs.get_data()

print(asdData.shape)
print(ncData.shape)

Reading /Users/wachiii/Workschii/brain-asd/data/filteredepoch/asd.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   10000.00 ms
        0 CTF compensation matrices available
Not setting metadata
957 matching events found
No baseline correction applied
0 projection items activated


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/2637720693.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-asd/data/filteredepoch/asd.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  asdEpochs = mne.read_epochs(asdfilteredEpochsPath)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/2637720693.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-asd/data/filteredepoch/nc.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ncEpochs = mne.read_epochs(ncfilteredEpochsPath)


Reading /Users/wachiii/Workschii/brain-asd/data/filteredepoch/nc.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   10000.00 ms
        0 CTF compensation matrices available
Not setting metadata
986 matching events found
No baseline correction applied
0 projection items activated
(957, 19, 2561)
(986, 19, 2561)


In [11]:
asdPsdObj = EEGPowerSpectrum(asdData, 256, ifNormalize=False)
asdPsdFeatures = asdPsdObj.run()

ncPsdObj = EEGPowerSpectrum(ncData, 256, ifNormalize=False)
ncPsdFeatures = ncPsdObj.run()

np.save(allFeaturesPath + "/psd/asdPsdFeatures.npy", asdPsdFeatures)
np.save(allFeaturesPath + "/psd/ncPsdFeatures.npy", ncPsdFeatures) 

### 2. Relative Power

In [12]:
import dotenv
import os
import sys
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from brainasd.features.psd import EEGPowerSpectrum


dotenv.load_dotenv()

asdfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/asd.fif"
ncfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/nc.fif"
allFeaturesPath = os.getenv("ALL_FEATURES_DIR_PATH")

NUM_CHANNELS = 19
EEG_SFREQ = 256


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }  

In [13]:
asdEpochs = mne.read_epochs(asdfilteredEpochsPath)
ncEpochs = mne.read_epochs(ncfilteredEpochsPath)

asdData = asdEpochs.get_data()
ncData = ncEpochs.get_data()

print(asdData.shape)
print(ncData.shape)

Reading /Users/wachiii/Workschii/brain-asd/data/filteredepoch/asd.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   10000.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/2965208461.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-asd/data/filteredepoch/asd.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  asdEpochs = mne.read_epochs(asdfilteredEpochsPath)


Not setting metadata
957 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-asd/data/filteredepoch/nc.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   10000.00 ms
        0 CTF compensation matrices available


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/2965208461.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-asd/data/filteredepoch/nc.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ncEpochs = mne.read_epochs(ncfilteredEpochsPath)


Not setting metadata
986 matching events found
No baseline correction applied
0 projection items activated
(957, 19, 2561)
(986, 19, 2561)


In [14]:
asdPsdObj = EEGPowerSpectrum(asdData, 256, ifNormalize=True)
asdPsdFeatures = asdPsdObj.run()

ncPsdObj = EEGPowerSpectrum(ncData, 256, ifNormalize=True)
ncPsdFeatures = ncPsdObj.run()

np.save(allFeaturesPath + "/relativepower/asdPsdFeatures.npy", asdPsdFeatures)
np.save(allFeaturesPath + "/relativepower/ncPsdFeatures.npy", ncPsdFeatures) 

### 3. Coherence

In [15]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainasd.features.coherence import EEGCoherence
import mne

dotenv.load_dotenv()


asdfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/asd.fif"
ncfilteredEpochsPath = os.getenv("FILTERED_EPOCHS_DIR_PATH") + "/nc.fif"
allFeaturesPath = os.getenv("ALL_FEATURES_DIR_PATH")

NUM_CHANNELS = 19
EEG_SFREQ = 256


# Declare Feature Dictionary
featureDict = {
    "coherence": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    } 


In [16]:
asdEpochs = mne.read_epochs(asdfilteredEpochsPath)
ncEpochs = mne.read_epochs(ncfilteredEpochsPath)

asdData = asdEpochs.get_data()
ncData = ncEpochs.get_data()

print(asdData.shape)
print(ncData.shape)

Reading /Users/wachiii/Workschii/brain-asd/data/filteredepoch/asd.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   10000.00 ms
        0 CTF compensation matrices available
Not setting metadata
957 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-asd/data/filteredepoch/nc.fif ...


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/2965208461.py:1: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-asd/data/filteredepoch/asd.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  asdEpochs = mne.read_epochs(asdfilteredEpochsPath)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_12845/2965208461.py:2: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-asd/data/filteredepoch/nc.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ncEpochs = mne.read_epochs(ncfilteredEpochsPath)


Isotrak not found
    Found the data of interest:
        t =       0.00 ...   10000.00 ms
        0 CTF compensation matrices available
Not setting metadata
986 matching events found
No baseline correction applied
0 projection items activated
(957, 19, 2561)
(986, 19, 2561)


In [17]:
asdCohObj = EEGCoherence(asdData, EEG_SFREQ)
asdCohFeatures = asdCohObj.run()
ncCohObj = EEGCoherence(ncData, EEG_SFREQ)
ncCohFeatures = ncCohObj.run()

np.save(allFeaturesPath + "/coherence/asdCohFeatures.npy", asdCohFeatures)
np.save(allFeaturesPath + "/coherence/ncCohFeatures.npy", ncCohFeatures)

## 10-folds CV ML

In [59]:
import os
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix

import dotenv
import warnings

dotenv.load_dotenv()
warnings.filterwarnings("ignore")


allFeatureDirPath = os.getenv("ALL_FEATURES_DIR_PATH")
modelCrossValDirPath = os.getenv("COHERENCE_FEATURE_CV_DIR_PATH")         # CHANGE PATH TO SAVE MODEL CROSS VALIDATION HERE!!!!


asdPsdFeatures = np.load(allFeatureDirPath + "/psd/asdPsdFeatures.npy")
ncPsdFeatures = np.load(allFeatureDirPath + "/psd/ncPsdFeatures.npy")
asdCohFeatures = np.load(allFeatureDirPath + "/coherence/asdCohFeatures.npy")
ncCohFeatures = np.load(allFeatureDirPath + "/coherence/ncCohFeatures.npy")
asdRelativepowerFeatures = np.load(allFeatureDirPath + "/relativepower/asdPsdFeatures.npy")
ncRelativepowerFeatures = np.load(allFeatureDirPath + "/relativepower/ncPsdFeatures.npy")

commonNumEpochs = min(asdPsdFeatures.shape[0], ncPsdFeatures.shape[0], 957)
def balanceData(features, numEpochs):
    return features[:numEpochs, :, :] 

asdPsdFeaturesBalanced = balanceData(asdPsdFeatures, commonNumEpochs)
ncPsdFeaturesBalanced = balanceData(ncPsdFeatures, commonNumEpochs)
asdCohFeaturesBalanced = balanceData(asdCohFeatures, commonNumEpochs)
ncCohFeaturesBalanced = balanceData(ncCohFeatures, commonNumEpochs)
asdRelativepowerFeaturesBalanced = balanceData(asdRelativepowerFeatures, commonNumEpochs)
ncRelativepowerFeaturesBalanced = balanceData(ncRelativepowerFeatures, commonNumEpochs)

print(f"Balanced asd PSD Features: {asdPsdFeaturesBalanced.shape}")
print(f"Balanced nc PSD Features: {ncPsdFeaturesBalanced.shape}")
print(f"Balanced asd Coherence Features: {asdCohFeaturesBalanced.shape}")
print(f"Balanced nc Coherence Features: {ncCohFeaturesBalanced.shape}")
print(f"Balanced asd Relative Power Features: {asdRelativepowerFeaturesBalanced.shape}")
print(f"Balanced nc Relative Power Features: {ncRelativepowerFeaturesBalanced.shape}")

Balanced asd PSD Features: (957, 5, 19)
Balanced nc PSD Features: (957, 5, 19)
Balanced asd Coherence Features: (957, 5, 171)
Balanced nc Coherence Features: (957, 5, 171)
Balanced asd Relative Power Features: (957, 5, 19)
Balanced nc Relative Power Features: (957, 5, 19)


In [60]:
paramGrid = {
    "C": [0.01, 0.1, 1],
    "kernel": ["linear", "rbf"],
    "gamma": ["scale", "auto"]
}

models = {
    "KNN": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7, 9], "weights": ["uniform", "distance"]}),
    "SVM": (SVC(probability=True, random_state=42), paramGrid),
    "Decision Tree": (DecisionTreeClassifier(), {"max_depth": [5, 10, 15]}),
    "Random Forest": (RandomForestClassifier(), {"n_estimators": [50, 100], "max_depth": [10, 20]}),
    "Logistic Regression": (LogisticRegression(), {"C": [0.01, 0.1, 1]})
}

featureSelectors = {
    "NoFeatureSelection": None,
    "SelectKBest": SelectKBest(score_func=f_classif, k=100),
    "VarianceThreshold": VarianceThreshold(threshold=0.01)  # Removes low variance features
    # "RFE": RFE(LogisticRegression(), n_features_to_select=100)
}

def get_stratified_kfold_data(X, y, nSplits=5):
    skf = StratifiedKFold(n_splits=nSplits, shuffle=True, random_state=42)
    return skf.split(X, y)

In [61]:
asdPsdFeaturesBalanced = asdPsdFeaturesBalanced.reshape(asdPsdFeaturesBalanced.shape[0], -1)
ncPsdFeaturesBalanced = ncPsdFeaturesBalanced.reshape(ncPsdFeaturesBalanced.shape[0], -1)
asdCohFeaturesBalanced = asdCohFeaturesBalanced.reshape(asdCohFeaturesBalanced.shape[0], -1)
ncCohFeaturesBalanced = ncCohFeaturesBalanced.reshape(ncCohFeaturesBalanced.shape[0], -1)
asdRelativepowerFeaturesBalanced = asdRelativepowerFeaturesBalanced.reshape(asdRelativepowerFeaturesBalanced.shape[0], -1)
ncRelativepowerFeaturesBalanced = ncRelativepowerFeaturesBalanced.reshape(ncRelativepowerFeaturesBalanced.shape[0], -1)

# asdFeatures = np.concatenate((asdPsdFeaturesBalanced, asdCohFeaturesBalanced, asdRelativepowerFeaturesBalanced), axis=1)
# ncFeatures = np.concatenate((ncPsdFeaturesBalanced, ncCohFeaturesBalanced, ncRelativepowerFeaturesBalanced), axis=1)

asdFeatures = asdCohFeaturesBalanced
ncFeatures = ncCohFeaturesBalanced   

X = np.concatenate((asdFeatures, ncFeatures), axis=0)
y = np.concatenate((np.ones(asdFeatures.shape[0]), np.zeros(ncFeatures.shape[0])), axis=0)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (1914, 855)
y shape: (1914,)


In [62]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

np.save(os.path.join(modelCrossValDirPath, "X_test.npy"), X_test)
np.save(os.path.join(modelCrossValDirPath, "y_test.npy"), y_test)

results = []

for modelName, (model, paramGrid) in models.items():
    for featureSelectorName, featureSelector in featureSelectors.items():
        print(f"Training {modelName} with {featureSelectorName}...")
        selector = featureSelector if featureSelector is not None else None
        
        foldAccuracies = []
        foldPrecisions = []
        foldRecalls = []
        foldF1Scores = []
        selectedFeatures = []
        foldMetrics = []

        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_fold = None
        best_f1_score = -np.inf
        best_fold_model = None

        for fold, (trainIdx, valIdx) in enumerate(skf.split(X_train_val, y_train_val)):
            XTrain, XVal = X_train_val[trainIdx], X_train_val[valIdx]
            yTrain, yVal = y_train_val[trainIdx], y_train_val[valIdx]

            if selector is not None:
                XTrain = selector.fit_transform(XTrain, yTrain)
                XVal = XVal[:, selector.get_support()]
            
            model.fit(XTrain, yTrain)
            
            yPred = model.predict(XVal)
            acc = accuracy_score(yVal, yPred)
            prec = precision_score(yVal, yPred)
            rec = recall_score(yVal, yPred)
            f1 = f1_score(yVal, yPred)
            
            foldAccuracies.append(acc) 
            foldPrecisions.append(prec) 
            foldRecalls.append(rec)  
            foldF1Scores.append(f1) 
            
            foldMetrics.append({
                "fold": fold + 1,
                "accuracy": acc,
                "precision": prec,
                "recall": rec,
                "f1_score": f1,
            })

            if selector is not None:
                selectedFeatures.append(np.where(selector.get_support())[0].tolist())

            model_dir = os.path.join(modelCrossValDirPath, f"{modelName}_{featureSelectorName}")
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)
            
            model_filename = os.path.join(model_dir, f"model_fold_{fold + 1}.pkl")
            joblib.dump(model, model_filename)
            print(f"Model for fold {fold + 1} saved to {model_filename}")

            if f1 > best_f1_score:
                best_f1_score = f1
                best_fold = fold
                best_fold_model = model

        print(f"Best fold: {best_fold + 1} with F1 score: {best_f1_score:.4f}")

        if selector is not None:
            X_train_val_selected = selector.fit_transform(X_train_val, y_train_val)
            X_test_selected = X_test[:, selector.get_support()]
        else:
            X_train_val_selected = X_train_val
            X_test_selected = X_test
        
        best_fold_model.fit(X_train_val_selected, y_train_val)
        yTestPred = best_fold_model.predict(X_test_selected)
        finalAccuracy = accuracy_score(y_test, yTestPred)
        finalPrecision = precision_score(y_test, yTestPred)
        finalRecall = recall_score(y_test, yTestPred)
        finalF1 = f1_score(y_test, yTestPred)

        # ROC Curve and Confusion Matrix
        y_prob = best_fold_model.predict_proba(X_test_selected)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))
        
        ax[0].plot(fpr, tpr, color='blue', label=f'ROC curve (area = {roc_auc:.2f}')
        ax[0].plot([0, 1], [0, 1], color='gray', linestyle='--')
        ax[0].set_xlabel('False Positive Rate')
        ax[0].set_ylabel('True Positive Rate')
        ax[0].set_title('ROC Curve')
        ax[0].legend(loc='lower right')
        
        cm = confusion_matrix(y_test, yTestPred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NC', 'ASD'], yticklabels=['NC', 'ASD'], ax=ax[1])
        ax[1].set_xlabel('Predicted Label')
        ax[1].set_ylabel('True Label')
        ax[1].set_title('Confusion Matrix')
        
        plot_filename = os.path.join(modelCrossValDirPath, f"{modelName}_{featureSelectorName}_ROC_ConfMatrix.png")
        plt.savefig(plot_filename)
        plt.close()

        print(f"Saved evaluation plots: {plot_filename}")

        # Save results into report
        results.append({
            "model": modelName,
            "feature_selection": featureSelectorName,
            "avg_accuracy": f"{np.mean(foldAccuracies):.4f} ± {np.std(foldAccuracies):.4f}",
            "avg_precision": f"{np.mean(foldPrecisions):.4f} ± {np.std(foldPrecisions):.4f}",
            "avg_recall": f"{np.mean(foldRecalls):.4f} ± {np.std(foldRecalls):.4f}",
            "avg_f1_score": f"{np.mean(foldF1Scores):.4f} ± {np.std(foldF1Scores):.4f}",
            "final_accuracy": f"{finalAccuracy:.4f}",
            "final_precision": f"{finalPrecision:.4f}",
            "final_recall": f"{finalRecall:.4f}",
            "final_f1_score": f"{finalF1:.4f}",
            "significant_features": np.unique([item for sublist in selectedFeatures for item in sublist]).tolist(),
            "folds_accuracy": foldAccuracies,
            "folds_precision": foldPrecisions,
            "folds_recall": foldRecalls,
            "folds_f1_score": foldF1Scores,
        })

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(modelCrossValDirPath, "results.csv"), index=False)
results_df.to_excel(os.path.join(modelCrossValDirPath, "results.xlsx"), index=False)

# Print results summary
for result in results:
    print(f"\n{result['model']} with {result['feature_selection']} - Final evaluation")
    print(f"Final Accuracy: {result['final_accuracy']}")
    print(f"Final Precision: {result['final_precision']}")
    print(f"Final Recall: {result['final_recall']}")
    print(f"Final F1 Score: {result['final_f1_score']}")
    print(f"Folds Accuracy: {result['folds_accuracy']}")
    print(f"Folds Precision: {result['folds_precision']}")
    print(f"Folds Recall: {result['folds_recall']}")
    print(f"Folds F1 Score: {result['folds_f1_score']}")


Training KNN with NoFeatureSelection...
Model for fold 1 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_1.pkl
Model for fold 2 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_2.pkl
Model for fold 3 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_3.pkl
Model for fold 4 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_4.pkl
Model for fold 5 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_5.pkl
Model for fold 6 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_6.pkl
Model for fold 7 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_7.pkl
Model for fold 8 saved to /Users/wachiii/Workschii/brain-asd/crossval/coherence/KNN_NoFeatureSelection/model_fold_8.pkl
